Les Misérables

## llm 설정

In [91]:
from langchain_community.chat_models import ChatOllama
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv,find_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatMessagePromptTemplate,ChatPromptTemplate,SystemMessagePromptTemplate,HumanMessagePromptTemplate,MessagesPlaceholder
from langchain import LLMChain
load_dotenv()

True

In [21]:
llm_model = ChatOllama(model='llama3.1:latest')
#response_llm = llm_model.invoke("Hello, how can I help you today?")
#print(response_llm.content)


In [22]:
gpt_model = ChatOpenAI(model_name="gpt-4")
#response_gpt = gpt_model.invoke("Hello, how can I help you today?")
#print(response_gpt.content)

## 1. llm 모델 작성

### 1-1. 프롬프트 작성

In [88]:
system_message = """
    너는 책을 읽어주는 어시스턴트야. 아래 지침에 따라 답변을 해줘야 해.
    모든 출력은 한국어로 해야 해.
    
    1. 사용자가 txt 파일을 제공하면, 해당 내용을 문단별로 출력해야 해.
    2. 사용자가 '다음'이라는 질문({question})을 하면, 다음 문단을 출력해야 해.
    3. txt 파일을 처음 받으면, {context_character}의 말투를 참고하여 등장인물처럼 대화해야 해.
    4. 문단 중에 " ", ' ' 와 같은 대화문이 나오면, 대화문 뒤에 : {emotion}을 함께 출력해야 해.
    5. 사용자가 책의 내용에 대해 질문({question})을 하면, 등장인물의 말투로 {context_books}를 참고하여 답변해야 해.
"""

In [92]:
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        MessagesPlaceholder(variable_name="history"), # 이전 대화 내용 저장용
        ("human", "{input}")
    ]
)
output_parser= StrOutputParser()#글자만 가져온다


### 1-2 대화 내용을 기억하는 llm 만들기

In [95]:
# 대화 메모리 생성 (대화 히스토리를 관리)
conversation_memory = ConversationBufferMemory()


# LLMChain 생성: 프롬프트와 LLM을 연결
llm_chain = LLMChain(
    llm=gpt_model,
    prompt=chat_prompt,
)

# 사용자 질문 처리 및 답변 생성 함수
def handle_user_input(user_input, history=[], context_books="책 내용 예시", context_character="주인공", emotion="감정 예시", question="질문 예시"):
    # 히스토리 업데이트
    history.append({"role": "user", "content": user_input})
    
    # 모델 실행 및 답변 생성
    response = llm_chain.run({
        "history": history,
        "input": user_input,
        "context_books": context_books,
        "context_character": context_character,
        "emotion": emotion,
        "question": question
    })
    
    # 히스토리에 응답 추가
    history.append({"role": "assistant", "content": response})
    
    return response, history

# 예시 실행
user_input = "첫 문단을 읽어줘"
history = []
response, history = handle_user_input(user_input, history)
print(response)


물론이죠, 첫 문단을 읽어드리겠습니다.

"어느 날, 한 소년이 집을 나서 그가 사랑하던 소녀를 만나러 가기로 했다. 그 소년의 이름은 밥이었다. 밥은 세상에서 가장 사랑스러운 소녀를 만나러 가는 길에 매우 행복했다. 그는 그녀에게 자신의 사랑을 고백할 계획이었다."


### 2. 텍스트 벡터화 임베딩

### 챗봇 ui

### 파일 업로드 처리

In [96]:
# 파일 업로드 처리 함수
# 이거 수정해서 벡터화 및 파일 db 생성해서 순서대로 부르게 하기
# gpt 보라
def chatbot_send_file(chat_history, file):
    if file is not None:
        with open(file.name, 'r', encoding='utf-8') as f:
            content = f.read()
        paragraphs = content.split('\n\n')  # 문단별로 분리
        ai_text = "\n\n".join(paragraphs)  # 문단을 다시 합쳐서 출력
        chat_history.append(["사용자가 업로드한 파일입니다.", ai_text])
    return chat_history

In [97]:
# 이벤트 처리 함수
def chatbot_send_message(chat_history, text):
    ai_text = llm_chain.invoke(text)  # 여기에 챗봇 처리 로직 추가
    chat_history.append([text, ai_text])
    return chat_history, ""

In [98]:
import gradio as gr


with gr.Blocks() as app:
    with gr.Tab("음성 인식봇"):
        with gr.Column():
            gr.Markdown(
                value="""
<center><h1> 음성 인식 봇 </h1></center>
<center> AI 인공지능 봇입니다. 음성으로 묻거나 문서 요약, 일정 관리 등을 할 수 있습니다.</center>
"""
            )
            cb_chatbot = gr.Chatbot(
                value=[[None, "안녕하세요 AI 챗봇입니다. 무엇이든 질문하세요"],
                       ['나는 지금 배가 고파요', '배가 고프시군요 메뉴를 추천해드릴까요?']
                       ],
                label="구미챗봇",
            )
        with gr.Row():
            cb_textbox = gr.Textbox(
                lines=3,
                placeholder="입력창입니다.",
                container=False,
                scale=7
            ) # 채팅 입력

            gr.Audio(
                format="mp3",
                scale=1,
                min_width=200,
                label="음성을 입력해주세요"
            ) # 음성 입력

            cb_send_btn = gr.Button(
                value="보내기",
                visible="primary",
                scale=1
            ) # 실행 버튼
            
            file_upload = gr.File(
                label="txt 파일을 업로드해주세요",
                type="file"
            ) # 파일 업로드

        with gr.Row():
            gr.Button(
                value="되돌리기"
            )
            gr.Button(
                value="초기화"
            )
        
        # 텍스트 입력 처리
        cb_send_btn.click(
            fn=chatbot_send_message,
            inputs=[cb_chatbot, cb_textbox],
            outputs=[cb_chatbot, cb_textbox]
        )
        
        # 파일 업로드 처리
        file_upload.change(
            fn=chatbot_send_file,
            inputs=[cb_chatbot, file_upload],
            outputs=[cb_chatbot]
        )

    with gr.Tab("문서 요약봇"):
        pass
    with gr.Tab("일정 관리봇"):
        pass

app.launch(debug=True)

Running on local URL:  http://127.0.0.1:7863
IMPORTANT: You are using gradio version 3.48.0, however version 4.29.0 is available, please upgrade.
--------

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
